In [6]:
import numpy as np
import pandas as pd
import h5py
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet
import time
from sklearn.svm import SVR
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import xgboost as xgb
import lightgbm as lgb
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
import matplotlib.pyplot as plt
import warnings

import logging
# 禁用 lightgbm 的警告信息
logging.getLogger('lightgbm').setLevel(logging.ERROR)

# 接下来可以继续正常使用 lightgbm

disease_mapping = {
    'control': 0,
    "Alzheimer's disease": 1,
    "Graves' disease": 2,
    "Huntington's disease": 3,
    "Parkinson's disease": 4,
    'rheumatoid arthritis': 5,
    'schizophrenia': 6,
    "Sjogren's syndrome": 7,
    'stroke': 8,
    'type 2 diabetes': 9
}
sample_type_mapping = {'control': 0, 'disease tissue': 1}


def load_idmap(idmap_dir):
    idmap = pd.read_csv(idmap_dir, sep=',')
    age = idmap.age.to_numpy()
    age = age.astype(np.float32)
    sample_type = idmap.sample_type.replace(sample_type_mapping)
    return age, sample_type


def load_methylation(methy_dir):
    '''
    Load methylation data from csv file.

    Note: We set nrows=5000 for test.
    If you want to use full data, it is recommended to read csv file by chunks 
      or other methods since the csv file is very large.
    Note the memory usage when you read csv file.

    We fill nan with 0, you can try other methods.
    '''
    methylation = pd.read_csv(methy_dir, sep=',', index_col=0, nrows=5000)
    methylation.fillna(0, inplace=True)
    methylation = methylation.values.T.astype(np.float32)
    return methylation


def load_methylation_h5(prefix):
    '''
    Load methylation data from .h5 file. 

    Parameters:
    ------------
    prefix: 'train' or 'test'
    '''
    methylation = h5py.File(prefix + '.h5', 'r')['data']
    h5py.File(prefix + '.h5', 'r').close()
    print("测试数据", len(methylation))
    print(len(methylation[:, :5000]))
#     return methylation[:, :5000]  # 5000 just for test
    return methylation[:, :]  # If you want to use full data, you can use this line.


# def train_ml(X_train, y_train):
#     model = ElasticNet()
#     model.fit(X_train, y_train)
#     return model

#############################################################################################


def train_random_forest(X_train, y_train):
    model = RandomForestRegressor()
    model.fit(X_train, y_train)
    return model

def train_gradient_boosting(X_train, y_train):
    model = GradientBoostingRegressor()
    model.fit(X_train, y_train)
    return model


def train_elastic(X_train, y_train):
    model = ElasticNet()
    print(model)
    model.fit(X_train, y_train)
    return model

def train_xgboost(X_train, y_train):
    model = xgb.XGBRegressor(objective='reg:squarederror')
    print(model)
    model.fit(X_train, y_train)
    return model

def train_adaboost(X_train, y_train):
    model = AdaBoostRegressor()
    print(model)
    model.fit(X_train, y_train)
    return model

def train_lgbm(X_train, y_train):
    model = lgb.LGBMRegressor()
    model.fit(X_train, y_train)

    return model

######################

def train_lgbm(X_train, y_train):
    # 设置模型参数
    params = {
        'num_leaves': 31,
        'max_depth': -1,
        'learning_rate': 0.1,
        'n_estimators': 300,
        'subsample': 1.0,
        'colsample_bytree': 1.0,
        'objective': 'regression',
        'metric': 'l1'
    }

    # 创建 LGBMRegressor 模型
    model = lgb.LGBMRegressor(**params)

    # 拟合模型
    model.fit(X_train, y_train)

    return model


# import lightgbm as lgb
# import numpy as np
# import matplotlib.pyplot as plt

# def custom_loss(y_true, y_pred):
#     # 自定义损失函数的计算逻辑（平方损失）
#     loss = np.square(y_true - y_pred).mean()
#     return loss

# def train_lgbm(X_train, y_train, num_iterations):
#     params = {
#         'objective': 'regression',  # 设置回归任务的目标函数类型
#         'metric': 'l2',  # 使用默认的均方差作为评估指标
#         'boosting_type': 'gbdt',
#         # 在这里设置其他参数
#     }
    
#     train_data = lgb.Dataset(X_train, label=y_train)
    
#     model = lgb.train(params, train_data, num_boost_round=num_iterations,
#                       valid_sets=[train_data], valid_names=['train'])
    
#     # 获取训练过程中的损失值
#     eval_results = model.evals_result_
#     train_loss = eval_results['train']['l2']
    
#     # 绘制损失曲线
#     plt.plot(train_loss, label='Train Loss')
#     plt.xlabel('Iterations')
#     plt.ylabel('Loss')
#     plt.legend()
#     plt.show()
    
#     return model


##############################################
# def train_loss(y_true, y_pred, sample_type):
#     mae_control = np.mean(np.abs(y_true[sample_type == 0] - y_pred[sample_type == 0])) # 对照组样本的平均绝对误差

#     case_true = y_true[sample_type == 1]
#     case_pred = y_pred[sample_type == 1]
#     above = np.where(case_pred >= case_true)
#     below = np.where(case_pred < case_true)

#     ae_above = np.sum(np.abs(case_true[above] - case_pred[above])) / 2
#     ae_below = np.sum(np.abs(case_true[below] - case_pred[below]))
#     mae_case = (ae_above + ae_below) / len(case_true)
#     mae = np.mean([mae_control, mae_case])
#     return mae


def train_ml(X_train, y_train, model_type):
    if model_type == 'lstm':
        model = train_lstm(X_train, y_train)
    elif model_type == 'xgboost':
        model = train_xgboost(X_train, y_train)
    elif model_type == 'adaboost':
        model = train_adaboost(X_train, y_train)
    elif model_type == 'lgbm':
        model = train_lgbm(X_train, y_train)
    elif model_type == "elastic":
        model = train_elastic(X_train, y_train)
    elif model_type == 'random_forest':  # 添加随机森林模型类型
        model = train_random_forest(X_train, y_train)
    elif model_type == 'gradient_boosting':  # 添加提升树模型类型
        model = train_gradient_boosting(X_train, y_train)
    else:
        raise ValueError("Invalid model type")
        
    return model


#######################################################################
def evaluate_ml(y_true, y_pred, sample_type):
    '''
    This function is used to evaluate the performance of the model. 

    Parameters:
    ------------
    y_true: true age
    y_pred: predicted age
    sample_type: sample type, 0 for control, 1 for case
    
    Return:
    ------------
    mae: mean absolute error.
    mae_control: mean absolute error of control samples. 对照组样本的平均绝对误差
    mae_case: mean absolute error of case samples.  病例组样本的平均绝对误差

    We use MAE to evaluate the performance.
    Please refer to evaluation section in the the official website for more details.
    '''
    mae_control = np.mean(np.abs(y_true[sample_type == 0] - y_pred[sample_type == 0])) # 对照组样本的平均绝对误差

    case_true = y_true[sample_type == 1]
    case_pred = y_pred[sample_type == 1]
    
    #################
#     def adjust_predicted_age(case_true,case_pred):
#         case_pred[case_pred < case_true] += 0.3
#         return case_pred
#33333333333333333333333333
    print(type(case_pred))
    print(case_pred.shape)

#     case_pred = adjust_predicted_age(case_true,case_pred)
    above = np.where(case_pred >= case_true)
    below = np.where(case_pred < case_true)

    ae_above = np.sum(np.abs(case_true[above] - case_pred[above])) / 2
    ae_below = np.sum(np.abs(case_true[below] - case_pred[below]))
    mae_case = (ae_above + ae_below) / len(case_true)

    mae = np.mean([mae_control, mae_case])
    return mae, mae_control, mae_case

In [ ]:
import warnings
warnings.filterwarnings("ignore")

# idmap_train_dir = '/trainmap.csv'
idmap_train_dir = '/mnt/Bioage/updated_trainmap.csv'

methy_train_dir = '/traindata.csv'
idmap_test_dir = '/mnt/Bioage/ai4bio_testset_final/testmap.csv'
methy_test_dir = '/mnt/Bioage/ai4bio_testset_final/testdata.csv'

age, sample_type = load_idmap(idmap_train_dir)

# Note: 'traindata.csv' is about 57GB, 'testdata.csv' is about 15GB.
# If you want to use h5 file,  you must run data_h5.py first to generate .h5 file.
# 'train.h5' is about 15GB, 'test.h5' is about 3.8GB.
# However, the memory usage is still large when you load .h5 file.
# Using this code directly on the free server provided by Tianchi will still
# result in insufficient memory when training ElasticNet on the full dataset.

use_h5 = True
if use_h5:
    methylation = load_methylation_h5('train')
    methylation_test = load_methylation_h5('test')
else:
    methylation = load_methylation(methy_train_dir)
    methylation_test = load_methylation(methy_test_dir)
print('Load data done')

indices = np.arange(len(age))
[indices_train, indices_valid, age_train, age_valid] = train_test_split(indices, age, test_size=0.3, shuffle=True) # random_state=42,

methylation_train, methylation_valid = methylation[indices_train], methylation[indices_valid]

sample_type_train, sample_type_valid = sample_type[indices_train], sample_type[indices_valid]

feature_size = methylation_train.shape[1]
del methylation
print('Start training...')
start = time.time()

model_type = 'lgbm'  # 可选择 'lstm', 'xgboost', 'adaboost', 'lgbm' 'elastic' 'random_forest   gradient_boosting


pred_model = train_ml(methylation_train, age_train, model_type)

# # 获取特征重要性得分
# feature_importance = pred_model.feature_importances_

# # 创建特征重要性的 DataFrame
# importance_df = pd.DataFrame({'Feature': X.columns, 'Importance': feature_importance})

# # 按重要性降序排序
# importance_df = importance_df.sort_values('Importance', ascending=False)

# # 打印特征重要性排名
# print(importance_df)

# pred_model = train_ml(methylation_train, age_train)
print(f'Training time: {time.time() - start}s')

age_valid_pred = pred_model.predict(methylation_valid)
mae = evaluate_ml(age_valid, age_valid_pred, sample_type_valid)
print(f'Validation MAE: {mae}')

age_pred = pred_model.predict(methylation_test)

age_pred[age_pred < 0] = 0  
# naive post-processing to ensure age >= 0

age_pred = np.around(age_pred, decimals=2)
age_pred = ['%.2f' % i for i in age_pred]
sample_id = pd.read_csv(idmap_test_dir, sep=',').sample_id
# Note: sample_id in submission should be the same as the order in testmap.csv.
# We do not provide the matching producdure for disordered sample_id in evaluation.

submission = pd.DataFrame({'sample_id': sample_id, 'age': age_pred})
submission_file = 'submit2.txt'
submission.to_csv(submission_file, index=False)

测试数据 8233
8233
测试数据 2063
2063
Load data done
Start training...


In [10]:
"""
xgboost
Training time: 11.390065670013428s
Validation MAE: (4.891016764822686, 4.7887278, 4.993305769330431)

Training time: 1117.8519587516785s
Validation MAE: (3.3071934799655844, 3.0595877, 3.5547992428748945)

adaboost:
Training time: 201.52103209495544s
Validation MAE: (9.747131501148274, 10.778602479313903, 8.715660522982644)

lgbm:
Training time: 7.726862668991089s
Validation MAE: (4.622818994742742, 4.831269383461401, 4.414368606024082)
LGBMRegressor()
Training time: 1762.3586609363556s
Validation MAE: (3.018452869747649, 2.943667307732819, 3.0932384317624786)

Training time: 1624.514690876007s
Validation MAE: (2.9165765886611785, 3.0095042118883777, 2.823648965433979)

elastic:
Training time: 10.774928569793701s
Validation MAE: (8.467154096179167, 9.084146, 7.850162646398862)
Training time: 1290.4735975265503s
Validation MAE: (4.642511094536491, 5.021964, 4.263058115891831)

gradient_boosting:
Training time: 481.118070602417s
Validation MAE: (6.093369791369678, 6.471409458326512, 5.715330124412843)

random_forest:
Training time: 1316.468659877777s
Validation MAE: (5.396996449669084, 5.10325217586691, 5.690740723471257)
"""

In [ ]:
"""
Validation MAE: (3.6879100562352765, 3.5677478235213966, 3.808072288949156)
(555,)

Validation MAE: (3.575001326467916, 3.3654093121172552, 3.7845933408185766)
"""

In [ ]:
import matplotlib.pyplot as plt

# 可视化验证集的实际年龄和预测年龄对比
plt.scatter(age_valid, age_valid_pred, c=sample_type_valid, cmap='viridis')
plt.plot([0, 120], [0, 120], 'r--')  # 理想情况下的对角线
plt.xlabel('Actual Age')
plt.ylabel('Predicted Age')
plt.title('Validation Set: Actual vs Predicted Age')
plt.colorbar(label='Sample Type')
plt.show()

In [12]:
print(methylation_train.shape)

(5763, 5000)


In [ ]:
import pandas as pd
import h5py


def savefile(methy_dir, chunk_size, name):
    df_chunks = pd.read_csv(methy_dir,
                            sep=',',
                            index_col=0,
                            chunksize=chunk_size)

    with h5py.File(name, 'w') as file:
        total_cols = 0
        for i, chunk in enumerate(df_chunks):
            chunk = chunk.transpose()
            chunk = chunk.fillna(0)
            # fill nan with 0, you can try other methods
            data_array = chunk.to_numpy()
            chunk_cols = data_array.shape[1]
            if i == 0:
                samples_num = data_array.shape[0]
                dataset = file.create_dataset('data',
                                              shape=data_array.shape,
                                              maxshape=(samples_num, None))

            dataset.resize((dataset.shape[0], total_cols + chunk_cols))

            dataset[:, total_cols:total_cols + chunk_cols] = data_array

            total_cols += chunk_cols  # Update total_cols within the loop

    return None


chunk_size = 5000

methy_train_dir = '/traindata.csv'
savefile(methy_train_dir, chunk_size, 'train.h5')
print('transform traindata over')

methy_test_dir = '/mnt/Bioage/ai4bio_testset_final/testdata.csv'
savefile(methy_test_dir, chunk_size, 'test.h5')
print('transform testdata over')